# Imports

In [1]:
import os
import json

import gandula

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Utils

In [10]:
def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def get_shoot_angle(x, y):
    _x = 52.5 - x
    return np.absolute(np.arctan((7.32 * _x) / (_x ** 2 + y ** 2 - (7.32 / 2)**2)))

def get_angle_from_player(p_ball, p_player):
    p_x = p_player[0]
    p_y = p_player[1]

    x = p_x - p_ball[0]
    y = p_y - p_ball[1]
    
    return np.absolute(np.arctan((1 * x) / (x ** 2 + y ** 2 - (1 / 2)**2)))

def get_triangle_area(pA, pB, pC):
    return np.absolute((pA[0]*(pB[1]-pC[1]) + pB[0]*(pC[1]-pA[1])+ pC[0]*(pA[1]-pB[1]))/2.0)

def isInside(t_p1, t_p2, t_p3, point):
    triangle_area = get_triangle_area(t_p1, t_p2, t_p3)
    area_1 = get_triangle_area(point, t_p2, t_p3)
    area_2 = get_triangle_area(t_p1, point, t_p3)
    area_3 = get_triangle_area(t_p1,t_p2, point)

    area_sum = area_1 + area_2 + area_3

    if(area_sum == triangle_area):
        return True
    else:
        return False

def get_players_location(frame):
    away_players = frame.away_players_with_kalman
    home_players = frame.away_players_with_kalman

    away_locations = []
    home_locations = []

    for player in away_players:
        x = player.x
        y = player.y

        away_locations.append((x, y))

    for player in home_players:
        x = player.x
        y = player.y

        home_locations.append((x, y))

    return home_locations, away_locations

def get_ball_location(frame):
    ball = frame.ball_with_kalman
    ball_x = ball.x
    ball_y = ball.y

    return (ball_x, ball_y)

def get_closest_player_from_all(home_locations, away_locations, target):
    
    is_home = True
    closest_distance = np.inf
    closest_index = -1

    for i, location in enumerate(home_locations):
        distance = euclidean_distance(location, target)
        if(distance < closest_distance):
            closest_distance = distance
            closest_index = i

    for i, location in enumerate(away_locations):
        distance = euclidean_distance(location, target)
        if(distance < closest_distance):
            closest_distance = distance
            closest_index = i
            is_home = False

    return closest_index, is_home


def get_closest_player(players_location, target):
    
    closest_index = -1
    closest_distance = np.inf

    for i, location in players_location:
        distance = euclidean_distance(location, target)
        if(distance < closest_distance):
            closest_distance = distance
            closest_index = i

    return closest_index


# Pre-processing

In [2]:
shot_frames = gandula.loader.read_pickle(
    'data/enhaced_frames_shots.pkl'
)

In [3]:
result = set()
# iterate over all frames to discover what event features are useful
for frame in shot_frames:
    event_features = list(frame['event'].shootingEvent.model_dump(
        exclude_none=True
    ).keys())
    result.update(event_features)

result

{'blockerPlayer',
 'clearerPlayer',
 'createsSpace',
 'deflectorBodyType',
 'deflectorPlayer',
 'keeperTouchType',
 'pressurePlayer',
 'pressureType',
 'saveHeightType',
 'saveReboundType',
 'saverPlayer',
 'shooterPlayer',
 'shotBodyType',
 'shotInitialHeightType',
 'shotNatureType',
 'shotOutcomeType',
 'shotPointX',
 'shotPointY'}

In [ ]:
def extract_features(raw_frames):
    """
    Extract features from raw data and returns the pandas DataFrame
    """
    data = []

    for frame in raw_frames:
        
        # general info
        ball_location = get_ball_location(frame) # ball_location is a tuple (x,y)
        home_locations, away_locations = get_players_location(frame)
        
        # who is attacking?
        closest_index, home_attacking = get_closest_player_from_all(home_locations, away_locations, ball_location)

        #dummy inits
        goalkeeper_location = (52.5, 0)
        num_blocking_players = 0
        blocking_angle = 0
        shoot_angle = 0
        # true inits
        shoot_angle = get_shoot_angle(ball_location[0], ball_location[1])

        if(home_attacking == True):
            # get goalkeeper location
            goalkeeper_location = get_closest_player(away_locations, (52.5, 0))
            # shoot_angle
            for away_pos in away_locations:
                # get num of blocking players
                if(isInside(ball_location, (52.5, -3.66), (52.5, 3.66), away_pos)):
                    num_blocking_players += 1
                    # get blocking angle
                    blocking_angle += get_angle_from_player(ball_location, away_pos)

        else:
            # get goalkeeper location
            goalkeeper_location = get_closest_player(home_locations, (52.5, 0))
            # shoot_angle
            for home_pos in home_locations:
                # get num of blocking players
                if(isInside(ball_location, (52.5, -3.66), (52.5, 3.66), home_pos)):
                    num_blocking_players += 1
                    # get blocking angle
                    blocking_angle += get_angle_from_player(ball_location, home_pos)
        
        goalkeeper_distance = euclidean_distance(ball_location, goalkeeper_location)
        goalkeeper_angle = get_angle_from_player(ball_location, goalkeeper_location)

        # set tracking features
        

        # set event features
        event_id            = frame['frame'].event_id
        frame_id            = frame['frame'].frame_id
        x_shot              = frame['event'].shootingEvent.shotPointX
        y_shot              = frame['event'].shootingEvent.shotPointY
        outcome             = frame['event'].shootingEvent.shotOutcomeType
        body_part           = frame['event'].shootingEvent.shotBodyType
        save_height         = frame['event'].shootingEvent.saveHeightType
        shot_initial_height = frame['event'].shootingEvent.shotInitialHeightType
        shooter_id          = frame['event'].shootingEvent.shooterPlayer.id
        






# EDA